# Fit NNs with/without Dropout to pure noise and inspect what they have learned.
We see that higher levels of Dropout preventing fitting to spurious high-order interaction effects.

Reproduces Figures 5 and C1 in the paper.

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import Model, layers
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

from purify import purify_3d
from xgboost import XGBRegressor as xgb

In [ ]:
n_samples = 1000
n_features = 25

X = np.random.uniform(-1, 1., size=(n_samples, n_features)).astype(np.float32)
Y = np.random.normal(0, 5., size=(n_samples, )).astype(np.float32)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

In [ ]:
def mean_square(y_pred, y_true):
    return tf.reduce_sum(tf.pow(y_pred-y_true, 2)) / (2 * n_samples)

def run_optimization(model, variables, optimizer):
    if variables is None:
        variables = model.trainable_variables
    with tf.GradientTape() as g:
        pred = model(X_train, training=True)
        loss = mean_square(pred, Y_train)
        gradients = g.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))

In [ ]:
def train(model, variables, optimizer,
          X_train, X_test, Y_train, Y_test,
          training_steps=10000, display_steps=50):
    prev_loss = np.inf
    for step in range(0, training_steps + 1):
        run_optimization(model, variables, optimizer)
        loss = mean_square(model(X_train, training=False), Y_train)
        if loss > prev_loss:
            return
        else:
            prev_loss = loss
        if step % display_steps == 0:
            loss = mean_square(model(X_train, training=False), Y_train)
            test_loss = mean_square(model(X_test, training=False), Y_test)
            print("Step: {:d}, loss: {:.5f}, test loss {:.5f}".format(step, loss, test_loss))

In [ ]:
class NeuralNet(Model):
    # Set layers.
    def __init__(self, dropout_p, weight_decay, n_hidden_1, n_hidden_2, n_hidden_3,
                 activation_dropout, input_dropout):
        super(NeuralNet, self).__init__()
        self.input_dropout = input_dropout    
        self.activation_dropout = activation_dropout
        # First fully-connected hidden layer.
        self.d0 = layers.Dropout(dropout_p)
        self.fc1 = layers.Dense(n_hidden_1, activation=tf.nn.relu,
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros')
        self.d1 = layers.Dropout(dropout_p)
        self.fc2 = layers.Dense(n_hidden_2, activation=tf.nn.relu,#.nn.relu,
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros')
        self.d2 = layers.Dropout(dropout_p)
        self.fc3 = layers.Dense(n_hidden_3, activation=tf.nn.relu,
                               kernel_initializer='glorot_uniform',
                               bias_initializer='zeros')
        self.d3 = layers.Dropout(dropout_p)
        self.out = layers.Dense(1, activation=tf.identity,
                                kernel_initializer='glorot_uniform',
                                bias_initializer='zeros')

    # Set forward pass.
    def call(self, x, training):
        if self.input_dropout:
            x0 = self.d0(x, training=training)
        else:
            x0 = x
        if self.activation_dropout:
            output = self.out(
                    self.d3(self.fc3(
                        self.d2(self.fc2(
                    self.d1(self.fc1(x0), training=training)),
                                training=training)),
                            training=training))
        else:
            output = self.out(self.fc3(
                self.fc2(self.fc1(x0))))
        return tf.squeeze(output)

In [ ]:
def append_or_new(dictionary, key, entry):
    try:
        dictionary[key].append(entry)
    except KeyError:
        dictionary[key] = [entry]

for n_hidden in [32]:#, 128]:
    for input_dropout in [True]:
        for activation_dropout in [True]:
            if activation_dropout == False and input_dropout == False:
                continue
            print(activation_dropout, input_dropout)
            n_fit_iters = 0
            n_hidden_1 = n_hidden
            n_hidden_2 = n_hidden
            n_hidden_3 = n_hidden
            dropout_ps = [0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75]
            for fit_iter in range(n_fit_iters):
                for dropout_p in dropout_ps:
                    for weight_decay in [0.0]:
                        print(dropout_p)
                        # Build a new model.
                        model = NeuralNet(dropout_p, weight_decay, n_hidden_1, n_hidden_2, n_hidden_3,
                                          activation_dropout, input_dropout)
                        optimizer = tf.optimizers.Adam(1e-3)

                        # Fit the model.
                        train(model, None, optimizer, 
                             X_train, X_test, Y_train, Y_test,
                             training_steps=10000, display_steps=100)

                        try:
                            models[dropout_p].append(model)
                        except KeyError:
                            models[dropout_p] = [model]

                        fig = plt.figure()
                        plt.hist(model(X_train, training=False))
                        plt.show()

            # Decompose the models.
            main_variances = {}
            pair_variances = {}
            three_variances = {}
            four_variances = {}
            good_dropout_ps = sorted(models.keys())
            for dropout_p in good_dropout_ps:
                for model in models[dropout_p]:
                    print(dropout_p)
                    pred = model(X, training=False).numpy()
                    xgb1 = xgb(max_depth=1, n_estimators=1000)
                    xgb2 = xgb(max_depth=2, n_estimators=1000)
                    xgb3 = xgb(max_depth=3, n_estimators=1000)

                    xgb1.fit(X, pred)
                    xgb1_preds = xgb1.predict(X)

                    xgb2.fit(X, pred)
                    xgb2_preds = xgb2.predict(X)

                    xgb3.fit(X, pred)
                    xgb3_preds = xgb3.predict(X)
                    residual3 = pred - xgb3_preds

                    append_or_new(main_variances, dropout_p, np.var(xgb1_preds))
                    append_or_new(pair_variances, dropout_p, np.var(xgb1_preds - xgb2_preds))
                    append_or_new(three_variances, dropout_p, np.var(xgb2_preds - xgb3_preds))
                    append_or_new(four_variances, dropout_p, np.var(residual3))


            fig = plt.figure(figsize=(8, 8))
            p0 = plt.errorbar(good_dropout_ps, np.array([np.mean(main_variances[p]) for p in good_dropout_ps]) / np.mean(main_variances[0]),
                         yerr=np.array([np.std(main_variances[p])/np.mean(main_variances[0]) for p in good_dropout_ps]),
                              label='$k=1$')
            p1 = plt.errorbar(good_dropout_ps, np.array([np.mean(pair_variances[p]) for p in good_dropout_ps]) / np.mean(pair_variances[0]),
                         yerr=np.array([np.std(pair_variances[p])/np.mean(pair_variances[0]) for p in good_dropout_ps]),
                             label='$k=2$')
            p2 = plt.errorbar(good_dropout_ps, np.array([np.mean(three_variances[p]) for p in good_dropout_ps]) / np.mean(three_variances[0]),
                         yerr=np.array([np.std(three_variances[p])/np.mean(three_variances[0]) for p in good_dropout_ps]),
                             label='$k=3$')
            p3 = plt.errorbar(good_dropout_ps, np.array([np.mean(four_variances[p]) for p in good_dropout_ps]) / np.mean(four_variances[0]),
                         yerr=np.array([np.std(four_variances[p])/np.mean(four_variances[0]) for p in good_dropout_ps]),
                             label='$k\geq 4$')

            plt.legend(fontsize=26)
            plt.xlabel('Dropout Rate', fontsize=32)
            plt.ylabel('Normalized Effect Size', fontsize=32)
            plt.xticks(fontsize=24)
            plt.yticks(fontsize=24)
            plt.tight_layout()
            plt.savefig('results/intro_fig_decay_{}_{}_{}.pdf'.format(n_hidden_1, activation_dropout, input_dropout),
                        bbox_inches='tight', dpi=300)

            fig = plt.figure(figsize=(8, 8))
            p0 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(main_variances[p]) for p in good_dropout_ps]),
                        yerr=np.array([np.std(main_variances[p]) for p in good_dropout_ps]))
            p1 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(pair_variances[p]) for p in good_dropout_ps]),
                   bottom=np.array([np.mean(main_variances[p]) for p in good_dropout_ps]),
                        yerr=np.array([np.std(pair_variances[p]) for p in good_dropout_ps]))
            p2 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(three_variances[p]) for p in good_dropout_ps]),
                   bottom=np.array([np.mean(main_variances[p]) for p in good_dropout_ps])
                         + np.array([np.mean(pair_variances[p]) for p in good_dropout_ps]),
                        yerr=np.array([np.std(three_variances[p]) for p in good_dropout_ps]))
            p3 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(four_variances[p]) for p in good_dropout_ps]),
                   bottom=np.array([np.mean(main_variances[p]) for p in good_dropout_ps])
                         + np.array([np.mean(pair_variances[p]) for p in good_dropout_ps])
                         + np.array([np.mean(three_variances[p]) for p in good_dropout_ps]),
                        yerr=np.array([np.std(four_variances[p]) for p in good_dropout_ps]))

            plt.xlabel("Dropout Rate", fontsize=32)
            plt.ylabel("Effect Size", fontsize=32)
            plt.xticks([i for i in range(len(dropout_ps))], dropout_ps, fontsize=24)
            plt.yticks(fontsize=24)
            plt.legend((p0[0], p1[0], p2[0], p3[0]), ('$k=1$', '$k=2$', '$k=3$', '$k\geq 4$'), fontsize=26)
            plt.tight_layout()
            plt.savefig('results/intro_fig_total_{}_{}_{}.pdf'.format(n_hidden_1, activation_dropout, input_dropout),
                        dpi=300, bbox_inches='tight')
            plt.show()

            fig = plt.figure(figsize=(8, 8))
            total_variances = {p : np.array(main_variances[p]) + np.array(pair_variances[p]) + np.array(three_variances[p]) + np.array(four_variances[p]) for p in good_dropout_ps}
            main_variances_normed = {p: (total_variances[p] > 2e-2)*main_variances[p] / total_variances[p] for p in good_dropout_ps}
            pair_variances_normed = {p: (total_variances[p] > 2e-2)*pair_variances[p] / total_variances[p] for p in good_dropout_ps}
            three_variances_normed = {p: (total_variances[p] > 2e-2)*three_variances[p] / total_variances[p] for p in good_dropout_ps}
            four_variances_normed = {p: (total_variances[p] > 2e-2)*four_variances[p] / total_variances[p] for p in good_dropout_ps}
            p0 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(main_variances_normed[p]) for p in good_dropout_ps]),
                        yerr=np.array([np.std(main_variances_normed[p]) for p in good_dropout_ps]))
            p1 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(pair_variances_normed[p]) for p in good_dropout_ps]),
                   bottom=np.array([np.mean(main_variances_normed[p]) for p in good_dropout_ps]),
                        yerr=np.array([np.std(pair_variances_normed[p]) for p in good_dropout_ps]))
            p2 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(three_variances_normed[p]) for p in good_dropout_ps]),
                   bottom=np.array([np.mean(main_variances_normed[p]) for p in good_dropout_ps])
                         + np.array([np.mean(pair_variances_normed[p]) for p in good_dropout_ps]),
                        yerr=np.array([np.std(three_variances_normed[p]) for p in good_dropout_ps]))
            p3 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(four_variances_normed[p]) for p in good_dropout_ps]),
                   bottom=np.array([np.mean(main_variances_normed[p]) for p in good_dropout_ps])
                         + np.array([np.mean(pair_variances_normed[p]) for p in good_dropout_ps])
                         + np.array([np.mean(three_variances_normed[p]) for p in good_dropout_ps]),
                        yerr=np.array([np.std(four_variances_normed[p]) for p in good_dropout_ps]))

            plt.xlabel("Dropout Rate", fontsize=32)
            plt.ylabel("Normalized Effect Size", fontsize=32)
            plt.xticks([i for i in range(len(good_dropout_ps))], good_dropout_ps, fontsize=24)
            plt.yticks(fontsize=24)
            plt.legend((p0[0], p1[0], p2[0], p3[0]), ('$k=1$', '$k=2$', '$k=3$', '$k\geq 4$'), fontsize=26)
            plt.tight_layout()
            plt.savefig('results/fixed_intro_fig_normed_{}_{}_{}.pdf'.format(n_hidden_1, activation_dropout, input_dropout),
                        bbox_inches='tight', dpi=300)
            plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
p0 = plt.errorbar(good_dropout_ps, np.array([np.mean(main_variances[p]) for p in good_dropout_ps]) / np.mean(main_variances[0]),
             yerr=np.array([np.std(main_variances[p])/np.mean(main_variances[0]) for p in good_dropout_ps]),
                  label='$k=1$')
p1 = plt.errorbar(good_dropout_ps, np.array([np.mean(pair_variances[p]) for p in good_dropout_ps]) / np.mean(pair_variances[0]),
             yerr=np.array([np.std(pair_variances[p])/np.mean(pair_variances[0]) for p in good_dropout_ps]),
                 label='$k=2$')
p2 = plt.errorbar(good_dropout_ps, np.array([np.mean(three_variances[p]) for p in good_dropout_ps]) / np.mean(three_variances[0]),
             yerr=np.array([np.std(three_variances[p])/np.mean(three_variances[0]) for p in good_dropout_ps]),
                 label='$k=3$')
p3 = plt.errorbar(good_dropout_ps, np.array([np.mean(four_variances[p]) for p in good_dropout_ps]) / np.mean(four_variances[0]),
             yerr=np.array([np.std(four_variances[p])/np.mean(four_variances[0]) for p in good_dropout_ps]),
                 label='$k\geq 4$')

plt.legend(fontsize=26)
plt.xlabel('Dropout Rate', fontsize=32)
plt.ylabel('Normalized Effect Size', fontsize=32)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.tight_layout()
plt.savefig('results/intro_fig_decay_{}_{}_{}.pdf'.format(n_hidden_1, activation_dropout, input_dropout),
            bbox_inches='tight', dpi=300)

fig = plt.figure(figsize=(8, 8))
p0 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(main_variances[p]) for p in good_dropout_ps]),
            yerr=np.array([np.std(main_variances[p]) for p in good_dropout_ps]))
p1 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(pair_variances[p]) for p in good_dropout_ps]),
       bottom=np.array([np.mean(main_variances[p]) for p in good_dropout_ps]),
            yerr=np.array([np.std(pair_variances[p]) for p in good_dropout_ps]))
p2 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(three_variances[p]) for p in good_dropout_ps]),
       bottom=np.array([np.mean(main_variances[p]) for p in good_dropout_ps])
             + np.array([np.mean(pair_variances[p]) for p in good_dropout_ps]),
            yerr=np.array([np.std(three_variances[p]) for p in good_dropout_ps]))
p3 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(four_variances[p]) for p in good_dropout_ps]),
       bottom=np.array([np.mean(main_variances[p]) for p in good_dropout_ps])
             + np.array([np.mean(pair_variances[p]) for p in good_dropout_ps])
             + np.array([np.mean(three_variances[p]) for p in good_dropout_ps]),
            yerr=np.array([np.std(four_variances[p]) for p in good_dropout_ps]))

plt.xlabel("Dropout Rate", fontsize=32)
plt.ylabel("Effect Size", fontsize=32)
plt.xticks([i for i in range(len(dropout_ps))], dropout_ps, fontsize=24)
plt.yticks(fontsize=24)
plt.legend((p0[0], p1[0], p2[0], p3[0]), ('$k=1$', '$k=2$', '$k=3$', '$k\geq 4$'), fontsize=26)
plt.tight_layout()
plt.savefig('results/intro_fig_total_{}_{}_{}.pdf'.format(n_hidden_1, activation_dropout, input_dropout),
            dpi=300, bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=(8, 8))
total_variances = {p : np.array(main_variances[p]) + np.array(pair_variances[p]) + np.array(three_variances[p]) + np.array(four_variances[p]) for p in good_dropout_ps}
main_variances_normed = {p: np.array(main_variances[p]) / total_variances[p] for p in good_dropout_ps}
pair_variances_normed = {p: pair_variances[p] / total_variances[p] for p in good_dropout_ps}
three_variances_normed = {p: three_variances[p] / total_variances[p] for p in good_dropout_ps}
four_variances_normed = {p: four_variances[p] / total_variances[p] for p in good_dropout_ps}
p0 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(main_variances_normed[p]) for p in good_dropout_ps]),
            yerr=np.array([np.std(main_variances_normed[p]) for p in good_dropout_ps]))
p1 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(pair_variances_normed[p]) for p in good_dropout_ps]),
       bottom=np.array([np.mean(main_variances_normed[p]) for p in good_dropout_ps]),
            yerr=np.array([np.std(pair_variances_normed[p]) for p in good_dropout_ps]))
p2 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(three_variances_normed[p]) for p in good_dropout_ps]),
       bottom=np.array([np.mean(main_variances_normed[p]) for p in good_dropout_ps])
             + np.array([np.mean(pair_variances_normed[p]) for p in good_dropout_ps]),
            yerr=np.array([np.std(three_variances_normed[p]) for p in good_dropout_ps]))
p3 = plt.bar(list(range(len(good_dropout_ps))), np.array([np.mean(four_variances_normed[p]) for p in good_dropout_ps]),
       bottom=np.array([np.mean(main_variances_normed[p]) for p in good_dropout_ps])
             + np.array([np.mean(pair_variances_normed[p]) for p in good_dropout_ps])
             + np.array([np.mean(three_variances_normed[p]) for p in good_dropout_ps]),
            yerr=np.array([np.std(four_variances_normed[p]) for p in good_dropout_ps]))

plt.xlabel("Dropout Rate", fontsize=32)
plt.ylabel("Normalized Effect Size", fontsize=32)
plt.xticks([i for i in range(len(good_dropout_ps))], good_dropout_ps, fontsize=24)
plt.yticks(fontsize=24)
plt.legend((p0[0], p1[0], p2[0], p3[0]), ('$k=1$', '$k=2$', '$k=3$', '$k\geq 4$'), fontsize=26)
plt.tight_layout()
plt.savefig('results/intro_fig_normed_{}_{}_{}.pdf'.format(n_hidden_1, activation_dropout, input_dropout),
            bbox_inches='tight', dpi=300)
plt.show()